In [ ]:
# This script obtains the open date for a given restaurant from Yelp
# Simple POC script for a few businesses

In [214]:
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests
import html
import datetime

In [215]:
df = pd.read_excel('yelp_data.xlsx')

In [216]:
def parse_base_url(url_string_with_params):
    return url_string_with_params.split('?')[0]
df['base_url'] = df['url'].apply(parse_base_url)

In [217]:
df['base_url'] = df['url'].apply(parse_base_url)

In [218]:
urls = list(df['base_url'])

In [239]:
def get_business_dates(base_url):
    """Obtain the date of the earliest reviews for a business on Yelp
    
    Args:
        base_url: Yelp business URL (no params)
        
    Returns a dictionary with the following keys:
        base_url: URL of the business
        open_date: Date of the earliest review on Yelp
    
    """
    
    # Earliest reviews
    querystring = {"sort_by":"date_asc"}
    response_rev = requests.request("GET", base_url, params=querystring).text
    
    #Parse response - review section varies slightly per page
    soup = BeautifulSoup(response_rev, 'html.parser')
    try:
        if 'aggregateRating' in soup.head.findAll('script')[6].string:
            reviews = json.loads(soup.head.findAll('script')[6].string)
        elif 'aggregateRating' in soup.head.findAll('script')[7].string:
            reviews = json.loads(soup.head.findAll('script')[7].string)
        elif 'aggregateRating' in soup.head.findAll('script')[8].string:
            reviews = json.loads(soup.head.findAll('script')[8].string)
        elif 'aggregateRating' in soup.head.findAll('script')[9].string:
            reviews = json.loads(soup.head.findAll('script')[9].string)
        elif 'aggregateRating' in soup.head.findAll('script')[10].string:
            reviews = json.loads(soup.head.findAll('script')[10].string)
        else:
            reviews = json.loads(soup.head.findAll('script')[11].string)
        
        first_review_date = reviews['review'][0]['datePublished']
    
    except:
        
        first_review_date = None

    # Create dict with results
    output = {}
    output['base_url'] = base_url
    output['open_date'] = first_review_date

    return output

In [234]:
df = pd.DataFrame(columns = ['base_url','open_date'])

In [242]:
for n, i in enumerate(urls[0:1]):
    print(n, i)
    df = df.append(get_business_dates(i), ignore_index=True)
    if n % 50 == 0:
        print("Writing to file")
        df.to_excel('yelp_dates_' + datetime.datetime.strftime(datetime.datetime.now(),'%Y%m%d%H%M%S') + '.xlsx')

0 https://www.yelp.com/biz/meline-food-center-brooklyn-2
Writing to file


In [240]:
get_business_dates('https://www.yelp.com/biz/tundra-deli-grocery-brooklyn')

{'base_url': 'https://www.yelp.com/biz/tundra-deli-grocery-brooklyn',
 'open_date': None}